In [ ]:
import pickle
# load the data
with open("pca_features.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features_pca = pickle.load(file)

In [2]:
import rtree
print(rtree.__version__)


1.3.0


In [5]:
print(len(track_ids))

11883


In [4]:
print((features_pca).shape)

(237660, 154)


# Construccion del index R-tree en vectores locales

In [ ]:
from rtree import index
from tqdm import tqdm

p = index.Property()
p.dimension = 154 
rtree_index = index.Index(properties=p)

total_vectors = len(track_ids) * 20  # Total de vectores para indexar
with tqdm(total=total_vectors, desc="Indexando vectores en R-Tree") as pbar:
    for song_idx, track_id in enumerate(track_ids):  # Iterando en las canciones
        for local_idx in range(20):  # Cada canción tiene 20 descriptores
            global_idx = song_idx * 20 + local_idx  # Índice global en features_pca
            vector = features_pca[global_idx]  # Obtener el vector local
            rtree_index.insert(global_idx, vector)  # Insertar en el R-Tree
            pbar.update(1)  # Incrementar la barra de progreso


Indexando vectores en R-Tree: 100%|██████████| 237660/237660 [14:16<00:00, 277.61it/s]


# Algoritmo KNN usando votación mayoritaria

In [ ]:
def knn_top_k_songs(rtree_index, features_pca, track_ids, query_vectors, k=2):
    """
    Realiza una búsqueda KNN en un R-Tree para 20 vectores de una canción y retorna las `k` canciones más cercanas.

    Parámetros:
        rtree_index (rtree.index.Index): Índice R-Tree construido con los vectores PCA.
        features_pca (np.ndarray): Matriz de vectores PCA.
        track_ids (list): Lista de IDs de canciones, uno por canción.
        query_vectors (np.ndarray): 20 vectores de consulta en el espacio PCA (de una canción).
        k (int): Número de canciones más cercanas a retornar.

    Retorna:
        list: Lista de las `k` canciones más cercanas, en formato:
              [(track_id1, votos1), (track_id2, votos2), ...].
    """
    from collections import Counter
    import numpy as np

    # Acumular resultados de los 20 vectores
    all_neighbors = []

    for query_vector in query_vectors:
        # Búsqueda KNN para cada vector de consulta
        nearest_neighbors = list(rtree_index.nearest(query_vector, num_results=20))  # Buscar 20 vecinos por vector ya que cada cancion tiene 20 vectores caracteristicos
        all_neighbors.extend(nearest_neighbors)

    # Asociar vecinos con sus canciones
    song_indices = [neighbor // 20 for neighbor in all_neighbors]  # Índices de canciones
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices]  # Mapear a track_ids

    # Votación mayoritaria
    song_counts = Counter(track_id_votes)
    top_songs = song_counts.most_common(k)  # Seleccionar las `k` canciones más votadas

    return top_songs


# Probando el KNN-Rtree

In [ ]:
print(track_ids[0]) #deberia de salir primero en la busqueda KNN

7pse475uICmWRY5hEkvPvI


In [ ]:
song_idx = 0
query_vectors = features_pca[song_idx * 20 : (song_idx + 1) * 20]
result = knn_top_k_songs(rtree_index, features_pca, track_ids, query_vectors, k=3) # Llamar a la función con k=3

# Mostrar resultados
for i, (song_id, votes) in enumerate(result, 1):
    print(f"Song {i}: '{song_id}', Votes: {votes}")

Song 1: '7pse475uICmWRY5hEkvPvI', Votes: 23
Song 2: '5CwOUooch74h0XarhDfAQK', Votes: 10
Song 3: '3bZCS8ThTAxMJZavYWOY1z', Votes: 8


1